In [ ]:
!pip install git+https://github.com/openai/whisper.git

In [ ]:
import whisper
import os
import requests
import json
from google.colab import files

# 定義 Gemini 摘要函數
def summarize_with_gemini(transcribed_text, api_key):
    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={api_key}"
    headers = {
        "Content-Type": "application/json"
    }
    payload = {
        "contents": [
            {
                "parts": [
                    {"text": f"Summarize the following meeting transcript into concise bullet points:\n\n{transcribed_text}"}
                ]
            }
        ]
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    if response.status_code == 200:
        result = response.json()
        try:
            # 修正解析邏輯，從 `candidates` 提取摘要
            summary = result["candidates"][0]["content"]["parts"][0]["text"]
            return summary
        except KeyError:
            print("Unexpected response format:", result)
            return None
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# 確保音頻文件是否存在，否則提示上傳
audio_path = "/content/water.m4a"

if not os.path.exists(audio_path):
    print("檔案 water.m4a 不存在，請上傳音頻文件。")
    uploaded = files.upload()
    audio_path = list(uploaded.keys())[0]  # 獲取上傳的檔案名稱

# 加載 Whisper 模型
model = whisper.load_model("medium")
result = model.transcribe(audio_path)
transcribed_text = result["text"]

# 打印逐字稿
print("Transcription:")
print(transcribed_text)

# 保存逐字稿為文本文件
file_name = os.path.splitext(os.path.basename(audio_path))[0]
with open(f"{file_name}.txt", "w") as file:
    file.write(transcribed_text)
    files.download(f"{file_name}.txt")  # 下載逐字稿

# 使用 Gemini API 生成摘要
api_key = "YOUR_API_KEY"  # 替換為您的 Gemini API 金鑰
summary = summarize_with_gemini(transcribed_text, api_key)

# 打印摘要
if summary:
    print("Meeting Summary:")
    print(summary)

    # 保存摘要為文本文件
    with open(f"{file_name}_summary.txt", "w") as file:
        file.write(summary)
        files.download(f"{file_name}_summary.txt")  # 下載摘要